In [102]:
data=DataFrame([np.random.random(100),np.random.random(100)]).transpose()
data.head()

,0,1
0,0.251575,0.145586
1,0.754694,0.506124
2,0.329115,0.484453
3,0.452784,0.879440
4,0.552003,0.994320


In [69]:
index=np.random.choice(100,1)

In [55]:
data.ix[index]

,0,1
87,0.711898,0.690416


In [139]:
#distance square
def dist_sq(a, b, axis = 1):
    return np.sum((a-b)**2,axis)
#distance square for every point to the centroid
def point_sq(data,index):
    Dist=np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        Dist[i]=(dist_sq(data.ix[index],data.ix[i]))
    return Dist
def dist_prob(Dist):
    return (Dist/np.sum(Dist))

In [140]:
def kmeansplusplus(data, k, d):
    centroid=np.zeros([k,d])
    index=np.random.choice(data.shape[0],1)
    centroid[0]=data.ix[index]
    for number in range(k-1):
        distance=point_sq(data,index)
        prob=dist_prob(distance).tolist()
        index=np.random.choice(data.shape[0],1,prob)
        centroid[number+1]=data.ix[index]
    return centroid
    

In [141]:
kmeansplusplus(data,9,2)

array([[ 0.55200343,  0.99432012],
       [ 0.98205991,  0.41942384],
       [ 0.23887628,  0.65181916],
       [ 0.80708619,  0.44687425],
       [ 0.61938776,  0.36537833],
       [ 0.37919413,  0.61969466],
       [ 0.92069586,  0.0866859 ],
       [ 0.34453112,  0.36503888],
       [ 0.43234641,  0.62702435]])

In [138]:
centroid=np.zeros([9,2])
index=np.random.choice(data.shape[0],1)
centroid[0]=data.ix[index]
data2=data
for number in range(9-1):
    data2=data2.drop(data2.index[index])
    distance=point_sq(data2,index)
    print(distance)
    prob=dist_prob(distance).tolist()
    index=np.random.choice(data2.shape[0],1,prob)
    centroid[number+1]=data2.ix[index]
centroid

KeyError: 67